# 性能基准测试

在大型项目中我们有可能会希望通过对比来查看某个功能的不同实现会有怎样不同的表现,这时候就可以使用`Benchmark.js`

安装:

    npm install --save-dev  benchmark
    
使用:

> 例:

我们知道js中字符串相连是有两种方式的:

In [1]:
"asdfg"+"hjkl"

'asdfghjkl'

In [2]:
"asdfg".concat("hjkl")

'asdfghjkl'

让我么来测测哪个快

待测代码 `strconcat.js`:

```js
export function str_plus(str1,str2) {
    return str1+str2
}
export function str_concat(str1,str2) {
    return str1.concat(str2)
}

```

测试代码`strconcat.benchmark.js`:

```js
import {str_plus,str_concat} from "./strconcat"
import Benchmark from 'benchmark'
let str1 = "asdfg"
let str2 = "hjkl"

let suite = new Benchmark.Suite
// 添加测试
suite
.add('+', function() {
  str_plus(str1,str2)
})
.add('concat', function() {
  str_concat(str1,str2)
})
// 每个测试跑完后，输出信息
.on('cycle', function(event) {
  console.log(String(event.target))
})
.on('complete', function() {
  console.log('Fastest is ' + this.filter('fastest').map('name'))
})
// 这里的 async 不是 mocha 测试那个 async 的意思，这个选项与它的时间计算有关，默认勾上就好了。
.run({ 'async': true })

```

之后再直接用`babel-node`运行即可

# 在交互环境中测试

我们同样可以在`babel-node`或者我们的`jupyter notebook`中做测试

In [3]:
function str_plus(str1,str2) {
    return str1+str2
}
function str_concat(str1,str2) {
    return str1.concat(str2)
}

In [4]:
import Benchmark from 'benchmark'
let str1 = "asdfg"
let str2 = "hjkl"

var suite = new Benchmark.Suite

try{
    // 添加测试
suite
.add('+', function() {
  str_plus(str1,str2)
})
.add('concat', function() {
  str_concat(str1,str2)
})
// 每个测试跑完后，输出信息
.on('cycle', function(event) {
  console.log(String(event.target))
})
.on('complete', function() {
  console.log('Fastest is ' + this.filter('fastest').map('name'))
})
// 这里的 async 不是 mocha 测试那个 async 的意思，这个选项与它的时间计算有关，默认勾上就好了。
.run({ 'async': true })
} catch(err) {
    console.log(err)
}


Suite {
  '0':
   Benchmark {
     name: '+',
     options:
      { async: false,
        defer: false,
        delay: 0.005,
        id: undefined,
        initCount: 1,
        maxTime: 5,
        minSamples: 5,
        minTime: 0,
        name: undefined,
        onAbort: undefined,
        onComplete: undefined,
        onCycle: undefined,
        onError: undefined,
        onReset: undefined,
        onStart: undefined },
     async: false,
     defer: false,
     delay: 0.005,
     initCount: 1,
     maxTime: 5,
     minSamples: 5,
     minTime: 0,
     id: 1,
     fn: [Function],
     stats:
      { moe: 0,
        rme: 0,
        sem: 0,
        deviation: 0,
        mean: 0,
        sample: [],
        variance: 0 },
     times: { cycle: 0, elapsed: 0, period: 0, timeStamp: 0 },
     _timerId:
      Timeout {
        _called: false,
        _idleTimeout: 5,
        _idlePrev: [TimersList],
        _idleNext: [TimersList],
        _idleStart: 11585,
        _onTimeout: [Functi

+ x 66,246,030 ops/sec ±2.41% (83 runs sampled)
concat x 70,009,103 ops/sec ±1.13% (90 runs sampled)
Fastest is concat


可见其实+运算符还是蛮好,而concat运行更快

## 内存使用情况


一般的后台开发语言中,内存使用的大小几乎没有限制.但是,V8最初是为浏览器打造的,在V8下64位系统可以操纵1.4GB内存,32位系统可以操纵0.7GB内存.在这样的限制下,node几乎不能直接操纵大内存.

node的process对象提供了对内存使用的快照接口`process.memoryUsage()`

In [11]:
process.memoryUsage()

{ rss: 31166464,
  heapTotal: 18690048,
  heapUsed: 13056152,
  external: 8859 }

其中
+ rss:resident set size，进程的常驻内存,是给这个进程分配了多少物理内存(占总分配内存的一部分) 这些物理内存中包含堆，栈，和代码段
+ heapTotal: 已经申请到的堆内存
+ heapUsed: 当前堆内存使用量
+ external: 代表V8管理的绑定到Javascript的C++对象的内存使用情况.

几乎和C语言一样,对象,字符串,闭包等存于堆内存.变量存于栈内存.实际的JavaScript源代码存于代码段内存.

In [12]:
process.memoryUsage()

{ rss: 31199232,
  heapTotal: 18690048,
  heapUsed: 13089784,
  external: 8895 }

因此要看一段代码用了多少内存,开头结尾处计算rss+heapUsed的差即可

In [15]:
let before = process.memoryUsage()
let before_used = before.rss+before.heapUsed
for (let i=0;i<10000;i++){
    
}
let after = process.memoryUsage()
let after_used = after.rss+after.heapUsed
console.log(after_used-before_used)

200
